In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import fetch_spot_history, historical_volatility
from src.mc_pricer import bs_price, mc_price_bs, mc_greeks_finite_differences, mc_delta_pathwise, mc_delta_lr
plt.style.use('seaborn')

In [ ]:
series = fetch_spot_history('SPY', period='2y', verbose=True)
S0 = float(series.iloc[-1])
sigma = historical_volatility(series)
r = 0.01
K = S0
T = 30/252  # ~30 calendar days
S0, sigma, r, K, T

In [ ]:
ns = [2000, 10000, 50000, 200000]
mc_prices = []
mc_ses = []
for n in ns:
    p,se,_ = mc_price_bs(S0, K, r, sigma, T, n_sims=n, option='call', seed=123, antithetic=True)
    mc_prices.append(p); mc_ses.append(se)
bs = bs_price(S0,K,r,sigma,T)
plt.errorbar(ns, mc_prices, yerr=[2*s for s in mc_ses], fmt='o-', label='MC +/- 2SE')
plt.axhline(bs, color='r', linestyle='--', label='BS price')
plt.xscale('log'); plt.xlabel('n_sims'); plt.ylabel('price'); plt.legend(); plt.title('MC convergence')

In [ ]:
n_sims = 100000
fd = mc_greeks_finite_differences(S0,K,r,sigma,T,n_sims=n_sims,option='call',bump_S=0.5,bump_sigma=1e-3,seed=123)
pw = mc_delta_pathwise(S0,K,r,sigma,T,n_sims=n_sims,option='call',seed=123)
lr = mc_delta_lr(S0,K,r,sigma,T,n_sims=n_sims,option='call',seed=123)
df = pd.DataFrame({
    'estimator': ['finite_diff','pathwise','likelihood_ratio'],
    'delta': [fd['delta'], pw['delta_pw'], lr['delta_lr']],
    'uncertainty_info': [fd['price_se'], pw['delta_pw_se'], lr['delta_lr_se']]
})
df

In [ ]:
p1,se1,_ = mc_price_bs(S0,K,r,sigma,T,n_sims=50000,seed=123,antithetic=False)
p2,se2,_ = mc_price_bs(S0,K,r,sigma,T,n_sims=50000,seed=123,antithetic=True)
print("No antithetic:", p1, "SE", se1)
print("With antithetic:", p2, "SE", se2)

In [1]:
import yfinance as yf
import pandas as pd
import sys, socket, ssl

print("yfinance version:", yf.__version__)
print("pandas version:", pd.__version__)
print("sys.executable:", sys.executable)

# Try fetching a tiny history
try:
    df = yf.download("SPY", period="5d", interval="1d", progress=False)
    print("Downloaded rows:", len(df))
    print(df.tail())
except Exception as e:
    print("Exception from yf.download():", type(e).__name__, e)

# alternative method
try:
    t = yf.Ticker("SPY")
    hist = t.history(period="5d")
    print("Ticker.history rows:", len(hist))
    print(hist.tail())
except Exception as e:
    print("Exception from Ticker.history():", type(e).__name__, e)

# Network basic check (DNS + TLS)
host = "query1.finance.yahoo.com"
try:
    addr = socket.gethostbyname(host)
    print(f"DNS OK: {host} -> {addr}")
    # quick TLS handshake check
    s = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=host)
    s.settimeout(3)
    s.connect((host, 443))
    print("TLS handshake OK")
    s.close()
except Exception as e:
    print("Network check failed:", type(e).__name__, e)

yfinance version: 0.2.65
pandas version: 2.2.3
sys.executable: /usr/local/bin/python3


/var/folders/zh/wylpw7053k13jb80qdlrcmj80000gn/T/ipykernel_10027/2936893739.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("SPY", period="5d", interval="1d", progress=False)


Downloaded rows: 5
Price            Close        High         Low        Open     Volume
Ticker             SPY         SPY         SPY         SPY        SPY
Date                                                                 
2025-11-21  659.030029  664.549988  650.849976  655.049988  123956200
2025-11-24  668.729980  670.059998  661.590027  662.690002   80437900
2025-11-25  675.020020  676.210022  664.479980  668.630005   81077100
2025-11-26  679.679993  681.700012  676.719971  677.630005   71879600
2025-11-28  683.390015  683.669983  680.500000  680.859985   49212000
Ticker.history rows: 5
                                 Open        High         Low       Close  \
Date                                                                        
2025-11-21 00:00:00-05:00  655.049988  664.549988  650.849976  659.030029   
2025-11-24 00:00:00-05:00  662.690002  670.059998  661.590027  668.729980   
2025-11-25 00:00:00-05:00  668.630005  676.210022  664.479980  675.020020   
2025-11-26 00